<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Capstone Project: Rossmann Drugstore

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

This machine learning project is taken from Kaggle. The link to the Kaggle competition page is https://www.kaggle.com/c/rossmann-store-sales

In their first Kaggle competition, Rossmann is challenging you to predict **7 weeks of daily sales for 1,115 stores** located across Germany. Reliable sales forecasts enable store managers to create effective staff schedules that increase productivity and motivation. By helping Rossmann create a robust prediction model, you will help store managers stay focused on what’s most important to them: their customers and their teams! 

# Datasets

1. train.csv - The train dataset, containing historical data including sales
2. store.csv - Supplemental information about the stores
3. test.csv - The test dataset, containing historical data excluding sales

## train.csv data fields

- Id - an Id that represents a (Store, Date) duple within the test set
- Store - a unique Id for each store
- DayOfWeek - 1 is for Monday, 2 is for Tuesday, and so on till 7 is for Sunday
- Sales - the turnover for any given day (this is what you are predicting)
- Customers - the number of customers on a given day
- Open - an indicator for whether the store was open: 0 = closed, 1 = open
- Promo - indicates whether a store is running a promo on that day
- StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
- SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools

## store.csv data fields

- Store - a unique Id for each store
- StoreType - differentiates between 4 different store models: a, b, c, d
- Assortment - describes an assortment level: a = basic, b = extra, c = extended
- CompetitionDistance - distance in meters to the nearest competitor store
- CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
- Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
- Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
- PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [2]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [10]:
# Loading the csv files into dataframes
types = {'StateHoliday': np.dtype(str)}

train = pd.read_csv(".\\train.csv", dtype=types, parse_dates=[2])
store = pd.read_csv(".\\store.csv")
test = pd.read_csv(".\\test.csv")

# Create train_sub and train_sub_validation data sets

We will be splitting the train dataset into two datasets.
1. The smaller of the two datasets is named **train_sub_validation**. It consists of the last 6 weeks of data (from 2018-06-20 to 2018-07-31) of the train dataset.

2. The larger of the two datasets is named **train_sub**. It consists of the remaining train dataset after splitting off train_sub_validation. We will be using this dataset to train our models.

In [3]:
# Split off the last 6 weeks of the time series data, and name it train_validation 
train_sub_validation = train[train['Date'] > '2015-06-19']
train_sub_validation.reset_index(drop=True,inplace=True)
train_sub_validation.to_csv('train_sub_validation.csv')

# The remaining data will be named train_sub
train_sub = train[train['Date'] <= '2015-06-19']
train_sub.reset_index(drop=True,inplace=True)
train_sub.to_csv('train_sub.csv')

# Fillna of 'Open' column in test dataset

There are 11 records in the test dataset where 'Open' is specified as NA.

We will assume that on these days, the store is open (fillna with 1), since none of these days are Sundays.

In [11]:
test.isnull().sum()

Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64

In [12]:
test[test['Open'].isnull()]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
479,480,622,4,2015-09-17,NaN,1,0,0
1335,1336,622,3,2015-09-16,NaN,1,0,0
2191,2192,622,2,2015-09-15,NaN,1,0,0
3047,3048,622,1,2015-09-14,NaN,1,0,0
4759,4760,622,6,2015-09-12,NaN,0,0,0
5615,5616,622,5,2015-09-11,NaN,0,0,0
6471,6472,622,4,2015-09-10,NaN,0,0,0
7327,7328,622,3,2015-09-09,NaN,0,0,0
8183,8184,622,2,2015-09-08,NaN,0,0,0
9039,9040,622,1,2015-09-07,NaN,0,0,0


In [13]:
# Replacing the NaN values in Open column with 1
test.fillna(value=1,axis=1,inplace=True)

# Export the test dataset and save over test.csv
test.to_csv('test.csv')